## Deployment Pipeline

> *This notebook works well with the `Data Science 3.0 (Python 3)` kernel on SageMaker Studio*

Use this notebook to process Legal Invoices

---
## Environment setup 

### SageMaker notebook permissions

▶️ In the [AWS IAM Console](https://console.aws.amazon.com/iamv2/home#/roles), check that you've attached the deployed OCR pipeline stack's **data science policy** to your SageMaker Execution Role, before continuing. You can find your deployed OCRPipeline stack in the [AWS CloudFormation Console](https://console.aws.amazon.com/cloudformation/home), and the Data Science Policy name is one of the Stack outputs.

### Notebook libraries and configurations

This notebook will require some additional libraries that aren't available by default in the SageMaker Studio Data Science kernel. Run the cell below to install the extra dependencies:

In [2]:
# Install Python libraries:
!pip install amazon-textract-response-parser \
    sagemaker-studio-image-build \
    "sagemaker>=2.87,<3"

# Install NodeJS:
NODE_VER = "v16.18.0"
NODE_DISTRO = "linux-x64"
!mkdir -p /usr/local/lib/nodejs
!wget -c https://nodejs.org/dist/{NODE_VER}/node-{NODE_VER}-{NODE_DISTRO}.tar.xz -O - | tar -xJ -C /usr/local/lib/nodejs
NODE_BIN_DIR = f"/usr/local/lib/nodejs/node-{NODE_VER}-{NODE_DISTRO}/bin"
ONPATH_BIN_DIR = "/usr/local/bin"
!ln -fs {NODE_BIN_DIR}/node {ONPATH_BIN_DIR}/node && \
    ln -fs {NODE_BIN_DIR}/npm {ONPATH_BIN_DIR}/npm && \
    ln -fs {NODE_BIN_DIR}/npx {ONPATH_BIN_DIR}/npx && \
    echo "NodeJS {NODE_VER} installed!"


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
--2023-05-26 22:30:04--  https://nodejs.org/dist/v16.18.0/node-v16.18.0-linux-x64.tar.xz
Resolving nodejs.org (nodejs.org)... 104.20.23.46, 104.20.22.46, 2606:4700:10::6814:172e, ...
Connecting to nodejs.org (nodejs.org)|104.20.23.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22496468 (21M) [application/x-xz]
Saving to: ‘STDOUT’

-                   100%[===================>]  21.45M  7.35MB/s    in 2.9s    

2023-05-26 22:30:07 (7.35 MB/s) - written to stdout [22496468/22496468]

NodeJS v16.18.0 installed!


With the extra libraries installed, you're ready to load them into the kernel and initialise clients for the various AWS services we'll be calling from the notebook:

In [3]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
from datetime import datetime
import json
from logging import getLogger
import os
import random
import re
import shutil
import time
from zipfile import ZipFile

# External Dependencies:
import boto3  # AWS SDK for Python
from IPython import display  # To display rich content in notebook
import pandas as pd  # For tabular data analysis
import sagemaker  # High-level SDK for SageMaker
from tqdm.notebook import tqdm  # Progress bars

# Local Dependencies:
#import util

# AWS service clients:
s3 = boto3.resource("s3")
smclient = boto3.client("sagemaker")
ssm = boto3.client("ssm")

logger = getLogger()

In [4]:
! pwd

/root/amazon-textract-transformer-pipeline1/notebooks


In [5]:
import sys

#sys.path.append('/root/BT-Labs/LayoutLM/version2/notebooks/util')
print(sys.path)

['/root/amazon-textract-transformer-pipeline1/notebooks', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/root/.ipython', '/root/BT-Labs/LayoutLM/version2/notebooks/util']


In [7]:
# Local Dependencies:
import util

This notebook will work with data sandboxes in Amazon S3, and connect to a deployed document processing pipeline solution. Below, we configure S3 data folders and read deployed pipeline parameter configuration from [AWS Systems Manager Parameter Store (AWS SSM)](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html):

In [44]:
# Use our own bucket loaded with training images 2023-05-23
#bucket_name = 'bt-digital-bt-labs-internal-200'
#bucket_prefix = "DynamicTableParser/4_TrainingData/"

bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "DynamicTableParser/"
account_id = sagemaker.Session().account_id()
region = os.environ["AWS_REGION"]

print ("Bucket Name: " + bucket_name)
print ("Bucket Prefix: " + bucket_prefix)
print ("account_id: " , account_id)
print ("region: ", region)
print()

# TO DO:  2023-05-27  TMP 
# we need a way to "remember" or identify the model we want to use.
# this code is OK but the lastest traning job is not always the model we want to run here
#training_job_name = estimator.latest_training_job.describe()["TrainingJobName"]
#  it seem we need a way to track the last training job FOR THIS APPLICATION, WORKFLOW, and DECISON
# we are working on.   We need a DB of keys that link the logical 
# APPLICATION, WORKFLOW, and DECISON to the "approved and ready to use" model we want to 
# use in PRODUCTION   2023-05-27  TMP 
training_job_name="ws-xlm-cfpb-hf-2023-05-26-11-15-49-226"   # hardcoding the name of the training job for now
print ("training_job_name: " , training_job_name)


Bucket Name: sagemaker-us-east-1-015943506230
Bucket Prefix: DynamicTableParser/
account_id:  015943506230
region:  us-east-1

training_job_name:  ws-xlm-cfpb-hf-2023-05-26-11-15-49-226


In [12]:
# S3 data locations:
# 2023-05-24
#bucket_name = sagemaker.Session().default_bucket()
#bucket_prefix = "textract-transformers-wshp/"

raw_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/raw-inference"
imgs_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/imgs-clean"
textract_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/textracted"
thumbs_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/thumbnails"
annotations_base_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/annotations"
print(f"Working in bucket s3://{bucket_name}/{bucket_prefix}\n")

try:
    config = util.project.init("ocr-transformers-demo")
    print(config)
except Exception as e:
    try:
        print(f"Your SageMaker execution role is: {sagemaker.get_execution_role()}")
    except Exception:
        print("Couldn't look up your SageMaker execution role")
    raise e

Working in bucket s3://sagemaker-us-east-1-015943506230/DynamicTableParser/

2023-05-27 12:24:44,419 project [INFO] Working in project 'ocr-transformers-demo'
<util.project.ProjectSession(
  project_id=ocr-transformers-demo,
  a2i_review_flow_arn_param=/ocr-transformers-demo/config/HumanReviewFlowArn,
  entity_config_param=/ocr-transformers-demo/config/EntityConfiguration,
  sagemaker_endpoint_name_param=/ocr-transformers-demo/config/SageMakerEndpointName,
  thumbnail_endpoint_name_param=/ocr-transformers-demo/config/ThumbnailEndpointName,
  a2i_execution_role_arn=arn:aws:iam::015943506230:role/OCRPipelineDemo-ProcessingPipelineReviewStepProces-M6UD1K31HQ0Q,
  pipeline_input_bucket_name=ocrpipelinedemo-pipelineinputbucket350ea1ae-1ffox60d5xk2z,
  model_callback_topic_arn=arn:aws:sns:us-east-1:015943506230:OCRPipelineDemo-ProcessingPipelineEnrichmentStepNLPEnrichmentModelSageMakerAsyncNLPEnrichmentModelE62625A1-WxGfSPiKRll3,
  model_results_bucket=ocrpipelinedemo-processingpipelineenric

## Part 1 - OCR the input PDFs

Use Textracks for the OCR conversions 

In [13]:
# build manifest.jsonl of raw input  files in the INBOX  

raw_bucket_name, raw_prefix = util.s3.s3uri_to_bucket_and_key(raw_s3uri)

valid_file_types = {"jpeg", "jpg", "pdf", "png", "tif", "tiff"}

n_files = 0
with open("data/raw-all.manifest.jsonl", "w") as f:
    # sorted() guarantees output order for reproducible sampling later:
    for obj in sorted(
        s3.Bucket(raw_bucket_name).objects.filter(Prefix=raw_prefix + "/"),
        key=lambda obj: obj.key,
    ):
        # Filter out any files you know shouldn't be counted:
        file_ext = obj.key.rpartition(".")[2].lower()
        if "/." in obj.key or file_ext not in valid_file_types:
            print(f"Skipping s3://{obj.bucket_name}/{obj.key}")
            continue

        # Save
        item = {"raw-ref": f"s3://{obj.bucket_name}/{obj.key}"}
        f.write(json.dumps(item)+"\n")
        n_files += 1

print(f"\nFound {n_files} valid files for OCR")


Skipping s3://sagemaker-us-east-1-015943506230/DynamicTableParser/data/raw-inference/

Found 9 valid files for OCR


In [14]:
%%time
textract_results = util.ocr.call_textract(
    textract_sfn_arn=config.plain_textract_sfn_arn,
    # Can instead use raw-all.manifest.jsonl to process whole dataset (see cost note above):
    input_manifest="data/raw-.manifest.jsonl",
    manifest_raw_field="raw-ref",
    manifest_out_field="textract-ref",
    # Map subpaths of {input_base} to subpaths of {output_base}:
    output_base_s3uri=textract_s3uri,
    input_base_s3uri=raw_s3uri,
    # Note that turning on additional features can have significant impact on API costs:
    features=["FORMS", "TABLES"],
    skip_existing=True,
)



CPU times: user 14.5 s, sys: 16.6 ms, total: 14.5 s
Wall time: 1min 24s


Once the extraction is done, write (only successful items) to a manifest file:

In [21]:
n_success = 0
n_fail = 0
with open("data/textracted-all.manifest.jsonl", "w") as fout:
    for ix, item in enumerate(textract_results):
        if isinstance(item["textract-ref"], str):
            fout.write(json.dumps(item) + "\n")
            n_success += 1
        else:
            if n_fail == 0:
                print ()
                print (item)
                print ()
                logger.error("First failure at index %s:\n%s", ix, item["textract-ref"])
                print ()
            n_fail += 1

print(f"{n_success} of {n_success + n_fail} docs processed successfully")
if n_fail > 0:
    raise ValueError(
        "Are you sure you want to continue? Consider re-trying to process the failed docs"
    )

9 of 9 docs processed successfully


---
## Part 2 - Extract clean input images (batch)

To annotate our documents with SageMaker Ground Truth image task UIs, we need **individual page images**, stripped of EXIF rotation metadata (because, at the time of writing, SMGT ignores this rotation for annotation consistency) and converted to compatible formats (since some formats like TIFF are not supported by most browsers).

For large corpora, this process of splitting PDFs and rotating and converting images may require significant resources - but is easy to parallelize.

Therefore instead of pre-processing the raw documents here in the notebook, this is a good use case for a scalable [SageMaker Processing Job](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html).

The job uses a **custom container image**, since the PDF reading tools we use aren't installed by default in pre-built SageMaker containers and aren't `pip install`able. However, the image has already been built and deployed to [Amazon Elastic Container Registry (ECR)](https://aws.amazon.com/ecr/) by the CDK stack (see `preproc_image` in [/pipeline/\_\_init\_\_.py](../pipeline/__init__.py)). All we need to do here is look it up from the stack parameters:

In [22]:
from sagemaker.processing import FrameworkProcessor, ProcessingInput, ProcessingOutput

ecr_image_uri = config.preproc_image_uri
print(f"Using pre-built custom container image:\n{ecr_image_uri}")
#015943506230.dkr.ecr.us-east-1.amazonaws.com/sm-ocr-preprocs:pytorch-1.10-inf-cpu

# Output S3 locations:
imgs_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/imgs-clean"
thumbs_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/thumbnails"

Using pre-built custom container image:
015943506230.dkr.ecr.us-east-1.amazonaws.com/sm-ocr-preprocs:pytorch-1.10-inf-cpu


> **Note:** The 'Non-augmented' manifest files used below for job data loading are still JSON-based, but a different format from the JSON-**Lines** manifests we use in most other places of this sample. You can find guidance on the [S3DataSource API doc](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3DataSource.html) for manifests as used here, and separate information in the [Ground Truth documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-input-data-input-manifest.html) on the "augmented" JSON-Lines manifests used elsewhere.

In [23]:
#### OPTION 1: For processing the whole raw_s3uri prefix:

preproc_inputs = [
     ProcessingInput(
         destination="/opt/ml/processing/input/raw",  # Expected input location, per our script
         input_name="raw",
         s3_data_distribution_type="ShardedByS3Key",  # Distribute between instances, if multiple
         source=raw_s3uri,  # S3 prefix for full raw document collection
     ),
 ]
print("Selected whole corpus -- OPTION 1")
#### END OPTION 1

Selected whole corpus -- OPTION 1


The cell below will **run the processing job** and show logs from the job as it progresses. You can also check up on the status and history of jobs in the [Processing page of the Amazon SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/processing-jobs).

> ⏰ **Note:** In our tests, it took (including job start-up overheads) about 8 minutes to process the 120-document sample with 2x `ml.c5.2xlarge` instances

In [24]:
%%time

processor = FrameworkProcessor(
    estimator_cls=util.preproc.DummyFramework,
    image_uri=ecr_image_uri,
    framework_version="",  # Not needed as image URI already provided
    base_job_name="ocr-img-dataclean",
    role=sagemaker.get_execution_role(),
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    volume_size_in_gb=15,
)

processor.run(
    code="preproc.py",  # PDF splitting / image conversion script
    source_dir="preproc",
    inputs=preproc_inputs[:],  # Either whole corpus or sample, as above
    outputs=[
        ProcessingOutput(
            destination=imgs_s3uri,
            output_name="imgs-clean",
            s3_upload_mode="Continuous",
            source="/opt/ml/processing/output/imgs-clean",  # Hi-res images for labelling
        ),
        ProcessingOutput(
            destination=thumbs_s3uri,
            output_name="thumbnails",
            s3_upload_mode="Continuous",
            source="/opt/ml/processing/output/thumbnails",  # Low-res images for model inputs
        ),
    ],
)

.........................2023-05-27 13:17:28,113 [preproc] INFO Parsed job args: Namespace(input='/opt/ml/processing/input/raw', n_workers=8, output='/opt/ml/processing/output/imgs-clean', thumbnails='/opt/ml/processing/output/thumbnails')
2023-05-27 13:17:28,113 [preproc] INFO Additional thumbnail output is ENABLED
2023-05-27 13:17:28,113 [preproc] INFO Reading raw files from /opt/ml/processing/input/raw
2023-05-27 13:17:28,113 [preproc] INFO Processing 5 files across 8 processes
2023-05-27 13:17:28,164 [preproc] INFO Parsed job args: Namespace(input='/opt/ml/processing/input/raw', n_workers=8, output='/opt/ml/processing/output/imgs-clean', thumbnails='/opt/ml/processing/output/thumbnails')
2023-05-27 13:17:28,164 [preproc] INFO Additional thumbnail output is ENABLED
2023-05-27 13:17:28,164 [preproc] INFO Reading raw files from /opt/ml/processing/input/raw
2023-05-27 13:17:28,165 [preproc] INFO Processing 4 files across 8 processes
2023-05-27 13:17:36,562 [preproc] INFO Processed doc 

Once the images have been extracted, we'll also **optionally** download them locally to the notebook for use in visualizations later:

In [25]:
print(f"Downloading cleaned images from {imgs_s3uri}...")
!aws s3 sync --quiet {imgs_s3uri} data/imgs-clean
print(f"Downloading thumbnail images from {thumbs_s3uri}...")
!aws s3 sync --quiet {thumbs_s3uri} data/imgs-thumb
print("Done")

Done


You'll see that this job also generates uniformly resized "thumbnail" images per page when the second (optional) `thumbnails` output is specified. These aren't important for the human annotation process, but will be used later for model training.

## Part 3 - Build Manifest

In [26]:
warnings = util.preproc.collate_data_manifest(
    # Output file:
    "data/pages-all-inference.manifest.jsonl",
    # Input manifest:
    #input_manifest="data/textracted-all.manifest2.jsonl",
    input_manifest="data/textracted-all.manifest.jsonl",
    # s3://... base URI used to try and map 'textract-ref's to cleaned images:
    textract_s3_prefix=textract_s3uri,
    # The s3://... base URI under which page images are stored:
    imgs_s3_prefix=imgs_s3uri,
    # Optional s3://... base URI also used to try and map 'raw-ref's to images if present:
    raw_s3_prefix=raw_s3uri,
    # Other output manifest settings:
    by="page",
    no_content="omit",
)

if len(warnings):
    raise ValueError(
        "Manifest usable but incomplete - %s docs failed. Please see `warnings` for details"
        % len(warnings)
    )

2023-05-27 13:25:17,368 trp [INFO] INFO: Detected K/V where key does not have content. Excluding key from output. 
Field
Key: 
Value: TBC_Insurance0014262 - {'BlockType': 'KEY_VALUE_SET', 'Confidence': 17.874217987060547, 'Geometry': {'BoundingBox': {'Width': 0.03270886838436127, 'Height': 0.010794537141919136, 'Left': 0.7914299964904785, 'Top': 0.9858063459396362}, 'Polygon': [{'X': 0.7914299964904785, 'Y': 0.9858063459396362}, {'X': 0.8241263628005981, 'Y': 0.9858697652816772}, {'X': 0.824138879776001, 'Y': 0.996600866317749}, {'X': 0.7914429903030396, 'Y': 0.9965370297431946}]}, 'Id': '288ba7d5-4b9d-492e-b3d0-d9bda975cdab', 'Relationships': [{'Type': 'VALUE', 'Ids': ['a5e2cf9e-f91c-4af0-977a-54ac221a5a5a']}], 'EntityTypes': ['KEY'], 'Page': 1}
2023-05-27 13:25:17,717 trp [INFO] INFO: Detected K/V where key does not have content. Excluding key from output. 
Field
Key: 
Value: TBC_Insurance0014278 - {'BlockType': 'KEY_VALUE_SET', 'Confidence': 16.114154815673828, 'Geometry': {'Boundin

## Part 4 - Easy "one-click" deployment

In [53]:
# Configure async endpoint settings for the pipeline stack:
async_inference_config = sagemaker.async_inference.AsyncInferenceConfig(
    output_path=f"s3://{config.model_results_bucket}",
    max_concurrent_invocations_per_instance=2,  # (Can tune this for performance)
    notification_config={
        "SuccessTopic": config.model_callback_topic_arn,
        "ErrorTopic": config.model_callback_topic_arn,
    },
)

# Extra environment variables to enable large payloads in async
async_extra_env_vars = {
    "MMS_DEFAULT_RESPONSE_TIMEOUT": str(60*3),  # 3min instead of default (maybe 60sec?)
    "MMS_MAX_REQUEST_SIZE": str(100*1024*1024),  # 100MiB instead of default ~6.2MiB
    "MMS_MAX_RESPONSE_SIZE": str(100*1024*1024),  # 100MiB instead of default ~6.2MiB
}

In [54]:
# If needed, you can attach to a previous training job by name like this:
print (training_job_name)
estimator = HuggingFaceEstimator.attach(training_job_name)

ws-xlm-cfpb-hf-2023-05-26-11-15-49-226

2023-05-26 11:41:47 Starting - Preparing the instances for training
2023-05-26 11:41:47 Downloading - Downloading input data
2023-05-26 11:41:47 Training - Training image download completed. Training in progress.
2023-05-26 11:41:47 Uploading - Uploading generated training model
2023-05-26 11:41:47 Completed - Training job completed


In [55]:
# Configurations:
hf_version = "4.17"
py_version = "py38"
pt_version = "1.10"
train_repo_name = "sm-ocr-training"
train_repo_tag = "hf-4.26-pt-gpu"  # (Base HF version is overridden in Dockerfile)
inf_repo_name = "sm-ocr-inference"
inf_repo_tag = train_repo_tag

# Combine together into the final URIs:
inf_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{inf_repo_name}:{inf_repo_tag}"
print(f"Target inference image: {inf_image_uri}")

Target inference image: 015943506230.dkr.ecr.us-east-1.amazonaws.com/sm-ocr-inference:hf-4.26-pt-gpu


In [56]:
predictor = estimator.deploy(
    # Avoid us accidentally deploying the same model twice by setting name per training job:
    endpoint_name=training_job_name,
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",  # Or try ml.m5.2xlarge
    image_uri=inf_image_uri,

    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),

    env={
        "PYTHONUNBUFFERED": "1",  # TODO: Disable once debugging is done
        "MMS_MAX_REQUEST_SIZE": str(100*1024*1024),  # Accept large payloads (docs)
        "MMS_MAX_RESPONSE_SIZE": str(100*1024*1024),  # Allow large responses
    },

    # Deploy in Asynchronous mode, to support large req/res payloads:
    async_inference_config=sagemaker.async_inference.AsyncInferenceConfig(
        output_path=f"s3://{config.model_results_bucket}",
        max_concurrent_invocations_per_instance=2,
        notification_config={
            "SuccessTopic": config.model_callback_topic_arn,
            "ErrorTopic": config.model_callback_topic_arn,
        },
    ),
)
print ("Async Endpoint ", training_job_name , " is InService.")
# TO DO: it would be cool to actually get the status of the endpoint from AWS and also display the creation date
# ARN and last updated date 

----------!

In [72]:
# Read the field lables we setup in the Ground Truth Job from a configuration file
# that was created previously. 
with open("data/field-config.json", "r") as f:
     data = json.load(f)
        
entity_classes = [f['Name'] for f in data]

# And print out a simple list:
print("\n".join(entity_classes))

Date
Person
Description
Hours
Rate
Total


## Part 5 - Inference 
For each page in our manifest we will send the Textracks .json output and thumbnail image to the model.  The model will then tag each WORD in the .json response with any identifed classes. 

In [73]:
with open("data/pages-all-inference.manifest.jsonl", "r") as fman:
    inference_examples = [json.loads(line) for line in filter(lambda l: l, fman)]

In [71]:
# this cell is only for testing.  If can be set to a RAW (not executable cell once you have 
# confidence the model is working as intented.


import ipywidgets as widgets
import trp

# Enabling thumbnails can significantly increase inference time here, but can improve results for
# models that consume image features (like LayoutLMv2, XLM):
include_thumbnails = False

def predict_from_manifest_item(
    item,
    predictor,
    imgs_s3key_prefix=imgs_s3uri[len("s3://"):].partition("/")[2],
    raw_s3uri_prefix=raw_s3uri,
    textract_s3key_prefix=textract_s3uri[len("s3://"):].partition("/")[2],
    imgs_local_prefix="data/imgs-clean",
    textract_local_prefix="data/textracted",
    draw=True,
):
    paths = util.viz.local_paths_from_manifest_item(
        item,
        imgs_s3key_prefix,
        textract_s3key_prefix=textract_s3key_prefix,
        imgs_local_prefix=imgs_local_prefix,
        textract_local_prefix=textract_local_prefix,
    )

    if include_thumbnails:
        doc_textract_s3key = item["textract-ref"][len("s3://"):].partition("/")[2]
        doc_raw_s3uri = raw_s3uri_prefix + doc_textract_s3key[len(textract_s3key_prefix):].rpartition("/")[0]
        print(f"Fetching thumbnails for {doc_raw_s3uri}")
        thumbs_async = preproc_predictor.predict_async(input_path=doc_raw_s3uri)
        thumbs_bucket, _, thumbs_key = thumbs_async.output_path[len("s3://"):].partition("/")
        # Wait for the request to complete:
        thumbs_async.get_result(sagemaker.async_inference.WaiterConfig())
        req_extras = {"S3Thumbnails": {"Bucket": thumbs_bucket, "Key": thumbs_key}}
        print("Got thumbnails result")
    else:
        req_extras = {}

    result_json = predictor.predict({
        "S3Input": {"S3Uri": item["textract-ref"]},
        "TargetPageNum": item["page-num"],
        "TargetPageOnly": True,
        **req_extras,
    })

    if "Warnings" in result_json:
        for warning in result_json["Warnings"]:
            logger.warning(warning)
    result_trp = trp.Document(result_json)

    if draw:
        util.viz.draw_smgt_annotated_page(
            paths["image"],
            entity_classes,
            annotations=[],
            textract_result=result_trp,
            # Note that page_num should be item["page-num"] if we requested the full set of pages
            # from the model above:
            page_num=1,
        )
    return result_trp


widgets.interact(
    lambda ix: predict_from_manifest_item(inference_examples[ix], predictor),
    ix=widgets.IntSlider(
        min=0,
        max=len(inference_examples) - 1,
        step=1,
        value=0,
        description="Example:",
    )
)

interactive(children=(IntSlider(value=0, description='Example:', max=181), Output()), _dom_classes=('widget-in…

<function __main__.<lambda>(ix)>

In [150]:
# run a prediction on page a a time

i = 1  # hardcoding to 1 for testing; need to read and prcess eveyr row in inference_examples

res = predict_from_manifest_item(
    inference_examples[i],
    predictor,
    draw=False,
)

2023-05-27 23:41:48,465 root [WARNING] SageMaker model's preprocessor (<class 'transformers.models.layoutxlm.processing_layoutxlm.LayoutXLMProcessor'>) expects page images (as .S3Thumbnails.{Bucket, Key} numpy array pointer in the request) but none were given. Generating default blank images - accuracy may be degraded.


In [151]:
def split_s3_path(s3_path):
    path_parts=s3_path.replace("s3://","").split("/")
    bucket=path_parts.pop(0)
    key="/".join(path_parts [:-1])
    filename = path_parts[-1]
    return bucket, key, filename

In [165]:
 def build_json_for_dataframe(entity_classes, text, confidence, label, label_confidence , top, left, textType,
                             filename, pagenumber,df):
  """
  Builds a JSON document from a DataFrame. The JSON document is then passed to Pandas to build a new DataFrame with the appended rows.

  """
  bucket, key, filename = split_s3_path(filename)

  # Create a new row with the `person`, `date`, and `rate` values.
  new_row = {"text": text, 
             "text_confidence": confidence, 
             "label": label, 
             "label_confidence": label_confidence, 
             "top": top,
             "left": left,          
             "top": top,     
             "textType" : textType,
             "file_name" : filename,
             "page_number" :  str(int(pagenumber)),
             "bucket" : bucket,
             "key" : key
            }

  # Append the new row to the DataFrame.
  new_df = df.append(new_row, ignore_index=True)

  # Return the new DataFrame.
  return new_df



In [166]:
print (entity_classes)

['Date', 'Person', 'Description', 'Hours', 'Rate', 'Total']


In [167]:
df = pd.DataFrame()

for page in res.pages:
    for line in page.lines:
        for word in line.words:
            label = ""
            label_confidence = "0.00"
            
            try:
                if ( int (word._block["PredictedClass"]) < int(len(entity_classes)) ):
                    label = (entity_classes[word._block["PredictedClass"]])
                else:
                    label =  "None"
                    label_confidence = "0.00"
            except: 
                label =  "None"
                label_confidence = "0.00"       
            
            new_df = build_json_for_dataframe(entity_classes,
                                              word.text, 
                                              word.confidence,
                                              label, 
                                              label_confidence,
                                              word.geometry.boundingBox.top,
                                              word.geometry.boundingBox.left,
                                              word.textType,
                                              inference_examples[i]['raw-ref'],
                                              inference_examples[i]['page-num'],
                                              df )
            df = new_df

print (len(df))


257


In [168]:
pd.set_option('max_rows', None)
df.head(100)

,text,text_confidence,label,label_confidence,top,left,textType,file_name,page_number,bucket,key
0,Sher,99.961830,None,0.00,0.081093,0.322362,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
1,Garner,99.975700,None,0.00,0.081046,0.395771,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
2,Cahill,99.927826,None,0.00,0.080641,0.498814,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
3,Richter,99.849930,None,0.00,0.080691,0.588421,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
4,Klein,99.951691,None,0.00,0.105747,0.349694,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
5,&,99.851669,None,0.00,0.105804,0.431569,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
6,"Hilbert,",99.933090,None,0.00,0.105484,0.462552,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
7,L.L.C.,95.647316,None,0.00,0.105493,0.576021,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
8,Twenty-Eighth,99.147263,None,0.00,0.127919,0.406729,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference
9,Floor,99.935326,None,0.00,0.128136,0.551785,PRINTED,Sher Garner Inv. 118116 (14262).pdf,2,sagemaker-us-east-1-015943506230,DynamicTableParser/data/raw-inference


In [52]:
# shut down end point after use so we don't incur needless charges. 
predictor.delete_endpoint(delete_endpoint_config=True)

## Part 6 - Parse inference response to a rows and columns (tabulear)

## Part 7 - Quality Control 
Extract control totals at the invoice level and compare the detail amounts of hours and total amount to the control totals


---

## Define the challenge

So we have our sample documents - what information would we like to extract from them?

As an example, we'll consider a market data aggregation use case: Collecting information like interest rates, fees, provider and product names, and some other more challenging examples like minimum payment descriptions and locally-applicable terms. The cell below defines the list of entities for the use-case, with some tips on how to annotate them that you'll also be able to see in the data labelling UI later:

In [ ]:
from util.postproc.config import FieldConfiguration

# For config API details, you can see the docs in the source file or run:
# help(FieldConfiguration)

fields = [
    # (To prevent human error, enter class_id=0 each time and update programmatically below)
    FieldConfiguration(0, "Date", optional=True, select="confidence",
        annotation_guidance=(
            "<p>This is the date that the task was performed by a person.  This should be a date field in mm/dd/yyyy, mm/dd, or yyyy-mm-dd, or dd-mm-yyyy</p>"
        ),
    ),
    FieldConfiguration(0, "Person", optional=True, select="confidence",
        annotation_guidance=(
            "<p>This is the person doing the work. It can have a column label such as Employee, Name, Timekeeper, Initals, etc.</p>"
        ),
    ),
    FieldConfiguration(0, "Description", optional=True, select="confidence",
        annotation_guidance=(
            "<p>It is a phrase or sentence that describes the task that was peformed by the person.</p>"
        ),
    ),
    FieldConfiguration(0, "Hours", optional=True, select="confidence",
        annotation_guidance=(
            "<p>This is a numeric or decimal value that represents the number of hours and minutes a person worked on the task.</p>"
        ),
    ),
    FieldConfiguration(0, "Rate", optional=True, select="confidence",
        annotation_guidance=(
            "<p>This is a numeric or decimal value that represents the amount the person charges per hour.</p>"
        ),
    ),
    FieldConfiguration(0, "Total", optional=True, select="confidence",
        annotation_guidance=(
            "<p>This is a numeric or deciaml amount that describes the total cost of Rate * Hours for a task.  It is optional.  This is not to be confused with a grand total. </p>"
        ),
    ),
    
]
for ix, cfg in enumerate(fields):
    cfg.class_id = ix

# Save the configuration to file:
with open("data/field-config.json", "w") as f:
    f.write(json.dumps(
        [cfg.to_dict() for cfg in fields],
        indent=2,
    ))

# And print out a simple list:
entity_classes = [f.name for f in fields]
print("\n".join(entity_classes))

---
## Filter a sample corpus

For a quick example model, there's no need for us to process or annotate all ~2,500 documents in the original corpus. Here, we'll select a random subset - but ensuring those present in the pre-prepared annotation data are kept:

In [ ]:
# Crawl source annotated Textract URIs from the job manifests:
annotated_textract_s3uris = util.ocr.list_preannotated_textract_uris(
    ann_jobs_folder="data/annotations/BT_annotations",
    exclude_job_names=["LICENSE"],
)
print (annotated_textract_s3uris)

In [ ]:


# Define how to check for matches:
def textract_uri_matches_doc_uri(tex_uri, doc_uri) -> bool:
    """Customize this function if needed for your use case's data layout"""
    # With our sample, Textract URIs will look like:
    # some/prefix/data/textracted/subfolders/file.pdf/consolidated.json
    tex_s3key = tex_uri[len("s3://"):].partition("/")[2]
    # With our sample, Raw URIs will look like:
    # some/prefix/data/raw/subfolders/file.pdf
    doc_s3key = doc_uri[len("s3://"):].partition("/")[2]

    # Given the expectations above:
    tex_rel_filepath = tex_s3key.partition("data/textracted/")[2].rpartition("/")[0]
    doc_rel_filepath = doc_s3key.partition("data/raw/")[2]
    return doc_rel_filepath == tex_rel_filepath

# Build the list of docs for which some annotations exist (prioritising debug over speed here):
annotated_doc_s3uris = set()
for uri in annotated_textract_s3uris:
    matching_doc_s3uris = [
        doc_s3uri
        for doc_s3uri in raw_doc_s3uris
        if textract_uri_matches_doc_uri(uri, doc_s3uri)
    ]
    n_matches = len(matching_doc_s3uris)
    if n_matches == 0:
        raise ValueError(
            "Couldn't find matching document in dataset for annotated Textract URI: %s"
            % (uri,)
        )
    if n_matches > 1:
        logger.warning(
            "Textract URI matched %s document URIs: Matching criterion may be too loose.\n%s\n%s",
            n_matches,
            uri,
            matching_doc_s3uris,
        )
    annotated_doc_s3uris.update(matching_doc_s3uris)

# This sorted list of required document S3 URIs is the main result you need to get to here:
annotated_doc_s3uris = sorted(annotated_doc_s3uris)
print(f"Found {len(annotated_doc_s3uris)} docs with pre-existing annotations")
print("For example:")
print("\n".join(annotated_doc_s3uris[:5] + ["..."]))

Both Amazon Textract and the multi-lingual entity recognition model we'll use later should be capable of processing Spanish, but you may want to exclude the small number of Spanish-language docs in the corpus if you're not able to confidently read and annotate them!

In [ ]:
N_DOCS_KEPT = 120
SKIP_SPANISH_DOCS = True


def include_filename(name: str) -> bool:
    """Filter out likely Spanish/non-English docs (if SKIP_SPANISH_DOCS enabled)"""
    if not name:
        return False
    if not SKIP_SPANISH_DOCS:
        return True
    name_l = name.lower()
    if (
        "spanish" in name_l
        or "tarjeta" in name_l
        or re.search(r"espa[nñ]ol", name_l)
        or re.search(r"[\[\(]esp?[\]\)]", name_l)
        or re.search(r"cr[eé]dito", name_l)
    ):
        return False
    return True


if N_DOCS_KEPT < len(annotated_doc_s3uris):
    raise ValueError(
        "Existing annotations cannot be used for model training unless the target documents are "
        "Textracted. To proceed with fewer docs than have already been annotated, you'll need to "
        "`exclude_job_names` per the 'data/annotations' folder (e.g. ['augmentation-1']) AND "
        "remember to not include them in notebook 2 (model training). Alternatively, increase "
        f"your N_DOCS_KEPT. (Got {N_DOCS_KEPT} vs {len(annotated_doc_s3uris)} prev annotations)."
    )

with open("data/raw-all.manifest.jsonl") as f:
    # First apply filtering rules:
    sampled_docs = [
        doc for doc in (json.loads(line) for line in f)
        if include_filename(doc["raw-ref"])
    ]

# Forcibly including the pre-annotated docs *after* the shuffling ensures that the order of
# sampling new docs is independent of what/how many have been pre-annotated:
required_docs = [d for d in sampled_docs if d["raw-ref"] in annotated_doc_s3uris]
random.Random(1337).shuffle(sampled_docs)
new_docs = [d for d in sampled_docs if d["raw-ref"] not in annotated_doc_s3uris]
sampled_docs = sorted(
    required_docs + new_docs[:N_DOCS_KEPT - len(required_docs)],
    key=lambda doc: doc["raw-ref"],
)

# Write the selected set to file:
with open("data/raw-sample.manifest.jsonl", "w") as f:
    for d in sampled_docs:
        f.write(json.dumps(d) + "\n")

print(f"Extracted random sample of {len(sampled_docs)} docs")
sampled_docs[:5] + ["..."]

> ▶️ In [data/raw-sample.manifest.jsonl](data/raw-sample.manifest.jsonl) you should now have an alphabetized list of the `N_DOCS_KEPT` randomly selected documents, which should include any documents referenced in existing annotations under `data/annotations`.

---
## OCR the input documents

> ⚠️ **Note:** Refer to the [Amazon Textract Pricing Page](https://aws.amazon.com/textract/pricing/) for up-to-date guidance before running large extraction jobs.
>
> At the time of writing, the projected cost (in `us-east-1`, ignoring free tier allowances) of analyzing 100 documents with 10 pages on average was approximately \\$67 with `TABLES` and `FORMS` enabled, or \\$2 without. Across the full corpus, we measured the average number of pages per document at approximately 6.7.

With (a subset of) the raw documents selected, the next ingredient is to link them with Amazon Textract-compatible OCR results in a new manifest - with entries something like:

```json
{"raw-ref": "s3://doc-example-bucket/folder/mydoc.pdf", "textract-ref": "s3://doc-example-bucket/folder/mydoc-textracted.json"}
```

We need to be mindful of the service [quotas](https://docs.aws.amazon.com/general/latest/gr/textract.html#limits_textract) when processing large batches of documents with Amazon Textract, to avoid excessive rate limiting and retries. Since an OCR pipeline solution stack is already set up for this sample, you can use just the *Amazon Textract portion of the pipeline* to process the documents in bulk.

> ⏰ This process took about 6 minutes to run against the 120-document sample set in our tests.

> ⚠️ **If you see errors in the output:**
>
> - Try re-running the cell - Rate limiting can sometimes cause intermittent failures, and the function will skip successfully processed files in repeat runs.
> - Persistent errors (on custom datasets) could be due to malformed files (remove them from the manifest) or very large files (see the [/CUSTOMIZATION_GUIDE.md](../CUSTOMIZATION_GUIDE.md) for tips on re-configuring your pipeline to handle very large documents).

In [ ]:
%%time
textract_results = util.ocr.call_textract(
    textract_sfn_arn=config.plain_textract_sfn_arn,
    # Can instead use raw-all.manifest.jsonl to process whole dataset (see cost note above):
    input_manifest="data/raw-sample.manifest.jsonl",
    manifest_raw_field="raw-ref",
    manifest_out_field="textract-ref",
    # Map subpaths of {input_base} to subpaths of {output_base}:
    output_base_s3uri=textract_s3uri,
    input_base_s3uri=raw_s3uri,
    # Note that turning on additional features can have significant impact on API costs:
    features=["FORMS", "TABLES"],
    skip_existing=True,
)

In [ ]:
# this is annoying.
# I am not sure where to look to "Doc failed to process - see results for details"
# this is just doing OCR with Textracks and writting to here
# maybe we can simulate this for now and use our own Textrack responses .json files 
# TO DO:  Use your own CODE TO OCR document to .JSON until we can figure out how to modify this code and view the log
print (textract_s3uri)
#  we are going to proceed with our .JSON files to bypass this step.

# fix:  use the default bucket.   I don't thing the the cloud formation template setup an ROLE with 
# to ready S3, just the default bucket.   
# What access role was created and if so can't we just update the rights to allow to read from S3?

# Workaround :   Use the Default S3 bucket - sagemaker-us-east-1-015943506230

In [ ]:
#print (textract_results)

Once the extraction is done, write (only successful items) to a manifest file:

In [ ]:
n_success = 0
n_fail = 0
with open("data/textracted-all.manifest.jsonl", "w") as fout:
    for ix, item in enumerate(textract_results):
        if isinstance(item["textract-ref"], str):
            fout.write(json.dumps(item) + "\n")
            n_success += 1
        else:
            if n_fail == 0:
                print ()
                print (item)
                print ()
                logger.error("First failure at index %s:\n%s", ix, item["textract-ref"])
                print ()
            n_fail += 1

print(f"{n_success} of {n_success + n_fail} docs processed successfully")
if n_fail > 0:
    raise ValueError(
        "Are you sure you want to continue? Consider re-trying to process the failed docs"
    )

> ▶️ You should now have a [data/textracted-all.manifest.jsonl](data/textracted-all.manifest.jsonl) JSON-Lines manifest file mapping source documents `raw-ref` to Amazon Textract result JSONs `textract-ref`: Both as `s3://...` URIs.

### Collate OCR and image data for annotation

Now we have a filtered corpus of documents with Amazon Textract results, plus cleaned and standardized images for each page - all available on Amazon S3.

To prepare for data annotation and later model training, we'll need to collate these together with a **page-level manifest** in JSON-lines format, with records something like:

```json
{"source-ref": "s3://doc-example-bucket/img-prefix/folder/filename-0001-01.png", "textract-ref": "s3://doc-example-bucket/tex-prefix/folder/filename.pdf/consolidated.json", "page-num": 1}
```

Key features of the format are:
- The `source-ref` is the path to a full-resolution cleaned page image (**not** a thumbnail), **but** model training in the next notebook will assume the equivalent thumbnail path is identical, except for some different s3://... bucket & prefix.
- The `page-num` is one-based (always >= 1), and for model training must match the image to the appropriate page number **in the linked Textract JSON file**.
    - For example if you have thumbnail `filename-0001-15.png` for page 15 of some long document, but for some reason your `textract-ref` JSON file contains *only* detections from page 15 of the document, you would set `"page-num": 1`.
- Mapping through the `raw-ref` here is nice to have, but optional, as the model training won't refer to the original document.

The key goal is to create a page-level catalogue that we're confident is correct, and for that reason the example function below will actually **validate that the artifacts are present on S3** in the expected locations.

> ⏰ Because of these validation checks, the cell below may a minute or two to run against our 120-document sample set.

In [ ]:
warnings = util.preproc.collate_data_manifest(
    # Output file:
    "data/pages-all-sample.manifest.jsonl",
    # Input manifest:
    input_manifest="data/textracted-all.manifest.jsonl",
    # s3://... base URI used to try and map 'textract-ref's to cleaned images:
    textract_s3_prefix=textract_s3uri,
    # The s3://... base URI under which page images are stored:
    imgs_s3_prefix=imgs_s3uri,
    # Optional s3://... base URI also used to try and map 'raw-ref's to images if present:
    raw_s3_prefix=raw_s3uri,
    # Other output manifest settings:
    by="page",
    no_content="omit",
)

if len(warnings):
    raise ValueError(
        "Manifest usable but incomplete - %s docs failed. Please see `warnings` for details"
        % len(warnings)
    )

> ▶️ You should now have a page-level catalogue linking `source-ref`, `textract-ref`, `page-num` in [data/pages-all-sample.manifest.jsonl](data/pages-all-sample.manifest.jsonl)

Let's briefly explore the catalogue we've created. Each line of the file is a JSON record identifying a particular page:

In [ ]:
with open("data/pages-all-sample.manifest.jsonl", "r") as f:
    for ix, line in enumerate(f):
        print(line, end="")
        if ix >= 2:
            print("...")
            break

The credit cards corpus has a very skewed distribution of number of pages per document, with a few outliers dragging up the average significantly. In our tests on corpus-wide statistics:

- The overall average was **~6.7 pages per document**
- The 25th percentile was 3 pages; the 50th percentile was 6 pages; and the 75th percentile was 11 pages
- The longest document was 402 pages

Your results for sub-sampled sets will likely vary - but can be analyzed as below:

In [ ]:
with open("data/pages-all-sample.manifest.jsonl", "r") as f:
    manifest_df = pd.DataFrame([json.loads(line) for line in f])
page_counts_by_doc = manifest_df.groupby("textract-ref")["textract-ref"].count()

print("Document page count statistics")
page_counts_by_doc.describe()

---
## Start the data labelling job

Now we have a correlated set of cleaned page images and OCR results for each page, we're ready to start annotating entities to collect model training data. Typically this is an iterative process with multiple rounds of labelling to balance experimentation speed with model accuracy. Here though, we'll show setting up a single small labelling job and combine the results with pre-existing annotations.

### Sample a dataset to label

Below, we:

- **Shuffle** our data (in a *reproducible*/deterministic way), to ensure we annotate documents/pages from a range of providers - not just concentrating on the first provider/doc(s)
- **Exclude** any examples for which the page image has **already been labeled** in the `data/annotations` output folder
- **Stratify** the sample, to obtain a specific (boosted) proportion of first-page samples, since we observed the first pages of documents to often be most useful for the fields of interest in the sample credit cards use case. (Many documents use the first page for a fact-sheet/summary, followed by subsequent pages of dense legal terms).

Run the cells below to select a small subset of previously-unlabelled pages and build a manifest file listing them:

In [ ]:
annotation_job_name = "cfpb-workshop-1"  # What will this job be called?
N_JOB_EXAMPLES = 15  # Select 15 new pages to annotate
PCT_FIRST_PAGE = .4  # 40% of samples should be page-num 1

preannotated_img_uris = [
    f"{imgs_s3uri}/{path}"
    for path in util.preproc.list_preannotated_img_paths(
        annotations_folder="data/annotations",
        exclude_job_names=[],
        key_prefix="data/imgs-clean/",
    )
]

job_input_manifest_file = f"data/manifests/{annotation_job_name}.jsonl"
os.makedirs("data/manifests", exist_ok=True)
print(f"'{annotation_job_name}' saving to: {job_input_manifest_file}")

with open(job_input_manifest_file, "w") as f:
    for ix, example in enumerate(
        util.preproc.stratified_sample_first_page_examples(
            input_manifest_path="data/pages-all-sample.manifest.jsonl",
            n_examples=N_JOB_EXAMPLES,  
            pct_first_page=PCT_FIRST_PAGE,
            exclude_source_ref_uris=preannotated_img_uris,
        )
    ):
        if ix < 3:
            print(example)
        elif ix == 3:
            print("...")
        f.write(json.dumps(example) + "\n")

To create the labelling job in SageMaker, this manifest file will also need to be uploaded to Amazon S3:

In [ ]:
input_manifest_s3uri = f"s3://{bucket_name}/{bucket_prefix}{job_input_manifest_file}"
!aws s3 cp $job_input_manifest_file $input_manifest_s3uri

### Create the labelling job

With a manifest file defining which pages should be included, and your "work team" already set up from earlier, you're ready to create your SageMaker Ground Truth labelling job.

You could also explore creating this via the AWS Console for SageMaker, but the code below will set up the job with the correct settings for you automatically:

In [ ]:
util.smgt.ensure_bucket_cors(bucket_name)

print(f"Starting labeling job {annotation_job_name}\non data {input_manifest_s3uri}\n")
create_labeling_job_resp = util.smgt.create_bbox_labeling_job(
    annotation_job_name,
    bucket_name=bucket_name,
    execution_role_arn=sagemaker.get_execution_role(),
    fields=fields,
    input_manifest_s3uri=input_manifest_s3uri,
    output_s3uri=annotations_base_s3uri,
    workteam_arn=workteam_arn,
    # To create a review/adjustment job from a manifest with existing labels in:
    # reviewing_attribute_name="label",
    s3_inputs_prefix=f"{bucket_prefix}data/manifests",
)
print(f"\nLABELLING JOB STARTED:\n{create_labeling_job_resp['LabelingJobArn']}")
print()
print(input_manifest_s3uri)
print(annotations_base_s3uri)
print(sagemaker.get_execution_role())
print("\n".join(["\nLabels:", "-------"] + entity_classes))

---
## Before you label - build custom containers

The entity recognition model we'll train later uses **customized containers**, which install extra libraries over the standard [SageMaker Hugging Face framework containers](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html).

> ⏰ Building these can take several minutes - so before you start labelling your documents in the SageMaker Ground Truth portal, **start the below cells running** to save some time.
>
> You don't need to wait for them to finish - just move on to the next "Label the data" section.

In [ ]:
# Configurations:
hf_version = "4.17"
py_version = "py38"
pt_version = "1.10"
train_repo_name = "sm-ocr-training"
#train_repo_tag = f"hf-{hf_version}-pt-gpu"
train_repo_tag = "hf-4.26-pt-gpu"  # (Base HF version is overridden in Dockerfile)
inf_repo_name = "sm-ocr-inference"
inf_repo_tag = train_repo_tag

account_id = sagemaker.Session().account_id()
region = os.environ["AWS_REGION"]

base_image_params = {
    "framework": "huggingface",
    "region": region,
    "instance_type": "ml.p3.2xlarge",  # (Just used to check whether GPUs/accelerators are used)
    "py_version": py_version,
    "version": hf_version,
    "base_framework_version": f"pytorch{pt_version}",
}

train_base_uri = sagemaker.image_uris.retrieve(**base_image_params, image_scope="training")
inf_base_uri = sagemaker.image_uris.retrieve(**base_image_params, image_scope="inference")

# Combine together into the final URIs:
train_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{train_repo_name}:{train_repo_tag}"
print(f"Target training image: {train_image_uri}")
inf_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{inf_repo_name}:{inf_repo_tag}"
print(f"Target inference image: {inf_image_uri}")

In [ ]:
%%time
# (No need to re-run this cell if your train image is already in ECR)

# Build and push the training image:
!cd custom-containers/train-inf && sm-docker build . \
    --compute-type BUILD_GENERAL1_LARGE \
    --repository {train_repo_name}:{train_repo_tag} \
    --role {config.sm_image_build_role} \
    --build-arg BASE_IMAGE={train_base_uri}

Note that although our training and inference containers use the [same Dockerfile](custom-containers/train-inf/Dockerfile), they're built from different parent images so both are needed in ECR:

In [ ]:
%%time
# (No need to re-run this cell if your inference image is already in ECR)

# Build and push the inference image:
!cd custom-containers/train-inf && sm-docker build . \
    --compute-type BUILD_GENERAL1_LARGE \
    --repository {inf_repo_name}:{inf_repo_tag} \
    --role {config.sm_image_build_role} \
    --build-arg BASE_IMAGE={inf_base_uri}

In [ ]:
# Check from notebook whether the images were successfully created:
ecr = boto3.client("ecr")
for repo, tag, uri in (
    (train_repo_name, train_repo_tag, train_image_uri),
    (inf_repo_name, inf_repo_tag, inf_image_uri)
):
    imgs_desc = ecr.describe_images(
        registryId=account_id,
        repositoryName=repo,
        imageIds=[{"imageTag": tag}],
    )
    assert len(imgs_desc["imageDetails"]) > 0, f"Couldn't find ECR image {uri} after build"
    print(f"Found {uri}")

---

## Label the data!

Shortly after the labeling job has been created, you'll see a new task for your user in the SageMaker Ground Truth **labeling portal**. If you lost the portal link from your email, you can access it from the *Private* tab of the [SageMaker Ground Truth Workforces console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-workforces).

▶️ Click **Start working** and annotate the examples until the all are finished and you're returned to the portal homepage.

▶️ **Try to be as consistent as possible** in how you annotate the classes, because inconsistent annotations can significantly degrade final model accuracy. Refer to the guidance (in this notebook and the 'Full Instructions') that we applied when annotating the example set.

![](img/smgt-task-pending.png "Screenshot of SMGT labeling portal with pending task")

### Sync the results locally (and iterate?)

Once you've finished annotating and the job shows as "Complete" in the [SMGT Console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-jobs) (which **might take an extra minute or two**, while your annotations are consolidated), you can download the results here to the notebook via the cell below:

In [ ]:
!aws s3 sync --quiet $annotations_base_s3uri ./data/annotations

You should see a subfolder created with the name of your annotation job, under which the **`manifests/output/output.manifest`** file contains the consolidated results of your labelling - again in the open JSON-Lines format.

▶️ **Check** your results appear as expected, and explore the file format.

> Because label outputs are in JSON-Lines, it's easy to consolidate, transform, and manipulate these results as required using open source tools!

---
## Consolidate annotated data

To construct a model training set, we'll typically need to consolidate the results of multiple SageMaker Ground Truth labelling jobs: Perhaps because the work was split up into more manageable chunks - or maybe because additional review/adjustment jobs were run to improve label quality.

Inside your `data/annotations` folder, you'll find some **pre-annotated augmentation data** provided for you already (in the `augmentation-` subfolders). These datasets are not especially large or externally useful, but will help you train an example model without too much (or even any!) manual annotation effort.

▶️ **Edit** the `include_jobs` line below to control which datasets (pre-provided and your own) will be included:

In [ ]:
include_jobs = [
    "layoutlm-boxes-2023-04-27-0251",
    "layoutlm-boxes-2023-05-02-0045",
    # TODO: Can edit the below to include your custom data, if you were able to label it:
    # "cfpb-workshop-1",
]


source_manifests = []
for job_name in sorted(filter(
    lambda n: os.path.isdir(f"data/annotations/BT_annotations/{n}"),
    os.listdir("data/annotations/BT_annotations")
)):
    if job_name not in include_jobs:
        logger.warning(f"Skipping {job_name} (not in include_jobs list)")
        continue
    job_manifest_path = f"data/annotations/BT_annotations/{job_name}/manifests/output/output.manifest"
    if not os.path.isfile(job_manifest_path):
        raise RuntimeError(f"Could not find job output manifest {job_manifest_path}")
    source_manifests.append({"job_name": job_name, "manifest_path": job_manifest_path})

print(f"Got {len(source_manifests)} annotated manifests:")
print("\n".join(map(lambda o: o["manifest_path"], source_manifests)))

Note that to **combine multiple output manifests to a single dataset**:

- The labels must be stored in the same attribute on every record (records use the labeling job name by default, which will be different between jobs).
- If importing data collected from some other account (like the `augmentation-` sets), we'll need to **map the S3 URIs** to equivalent links on your own bucket.

In [ ]:
standard_label_field = "label"

print("Writing data/annotations/annotations-all.manifest.jsonl")
with open("data/annotations/annotations-all.manifest.jsonl", "w") as fout:
    util.preproc.consolidate_data_manifests(
        source_manifests,
        fout,
        standard_label_field=standard_label_field,
        bucket_mappings={"DOC-EXAMPLE-BUCKET": bucket_name},
        prefix_mappings={"EXAMPLE-PREFIX/": bucket_prefix},
    )

In [ ]:
def split_s3_path(s3_path):
    path_parts=s3_path.replace("s3://","").split("/")
    bucket=path_parts.pop(0)
    key="/".join(path_parts [:-1])
    filename = path_parts[-1]
    return bucket, key, filename

In [ ]:
# populate the dictionary of our annotation manafests created in Gound Truth 
annotation_manafest_dict = {}
with open("data/annotations/annotations-all.manifest.jsonl", "r") as f:
    for line in f:
        annotation_json = json.loads(line)
        bucket, key, filename = split_s3_path(  annotation_json['source-ref'])

        # update the soure ref to our use our thumbnails we created.
        # this is useful if you use LayoutLM2 and beyond
        new_source_ref = r's3://' + bucket_name + r'/' + bucket_prefix + r'data/thumbnails/' + filename
        annotation_json['source-ref'] = new_source_ref
        #print (annotation_json)
        annotation_manafest_dict[filename] = annotation_json
        

In [ ]:
# enhance the textracks json that we will use for training data; we need
# to add our Ground Truth annatations 
with open("data/textracted-BT-enhanced.manifest.jsonl", "w") as updated_json:
    with open("data/textracted-all.manifest.jsonl", "r") as f:
        for line in f:
            texttracts_json = json.loads(line)
            bucket, key, filename = split_s3_path( texttracts_json['raw-ref'])
            #print (bucket)
            #print (key)     
            #print (filename)

            # get the annotation json that matches the texttrack source file name
            annotation_json = annotation_manafest_dict[filename]

            # add the attributres form the annotation json to the text track json 
            texttracts_json['source-ref'] = annotation_json['source-ref']
            texttracts_json['label'] = annotation_json['label'] 
            #print (texttracts_json)
            #print ()
            
            json.dump(texttracts_json, updated_json)
            updated_json.write("\n")
            

### Split training and test sets

To get some insight on how well our model is generalizing to real-world data, we'll need to reserve some annotated data as a testing/validation set.

Below, we randomly partition the data into training and test sets and then upload the two manifests to S3:

In [ ]:
# TO DO:  Can we build a  "data/annotations/annotations-all.manifest.jsonl" with the .json docs?


def split_manifest(f_in, f_train, f_test, train_pct=0.9, random_seed=1337):
    """Split `f_in` manifest file into `f_train`, `f_test`"""
    logger.info(f"Reading {f_in}")
    with open(f_in, "r") as fin:
        lines = list(filter(lambda line: line, fin))
    logger.info("Shuffling records")
    random.Random(random_seed).shuffle(lines)
    n_train = round(len(lines) * train_pct)

    with open(f_train, "w") as ftrain:
        logger.info(f"Writing {n_train} records to {f_train}")
        for line in lines[:n_train]:
            
            #line = line.replace('source-ref','textract-ref')
            #line = line.replace('.jpeg','.json')
            #line = line.replace('DynamicTableParser/4_TrainingData/Cycle-1','DynamicTableParser/data/textracted')           
            #line = line.replace('DynamicTableParser/4_TrainingData/Cycle-0','DynamicTableParser/data/textracted')           
                                   
            #print (line)
            #print ()
            #print ()
            
            ftrain.write(line)
            
    with open(f_test, "w") as ftest:
        logger.info(f"Writing {len(lines) - n_train} records to {f_test}")
        for line in lines[n_train:]:
            templine = line
            
            #line = line.replace('source-ref','textract-ref')
            #line = line.replace('.jpeg','.json')
            #line = line.replace('DynamicTableParser/4_TrainingData/Cycle-1','DynamicTableParser/data/textracted')           
            #line = line.replace('DynamicTableParser/4_TrainingData/Cycle-0','DynamicTableParser/data/textracted')           
            
            #print ()
            #print (templine)
            #print ()
            #print (line)
            #print ()
            #print ()
            
            ftest.write(line)
            


split_manifest(
   # "data/annotations/annotations-all.manifest.jsonl",
    "data/pages-all-sample.manifest.jsonl",
    "data/annotations/annotations-train.manifest.jsonl",
    "data/annotations/annotations-test.manifest.jsonl",

)

In [ ]:
train_manifest_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/annotations/annotations-train.manifest.jsonl"
!aws s3 cp data/annotations/annotations-train.manifest.jsonl $train_manifest_s3uri

print ()
test_manifest_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/annotations/annotations-test.manifest.jsonl"
!aws s3 cp data/annotations/annotations-test.manifest.jsonl $test_manifest_s3uri

### Visualize the data

Before training the model, we'll sense-check the data by plotting a few examples.

The utility function below will overlay the page image with the annotated bounding boxes, the locations of `WORD` blocks detected from the Amazon Textract results, and the resulting classification of individual Textract `WORD`s. To render these results, the Amazon Textract OCR results need to be downloaded locally to the notebook:

In [ ]:
%%time

!aws s3 sync --quiet $textract_s3uri ./data/textracted

In [ ]:
with open("data/annotations/annotations-test.manifest.jsonl", "r") as fman:
    test_examples = [json.loads(line) for line in filter(lambda l: l, fman)]

---
## Train the entity recognition model

We now have all the data needed to train and validate an layout- and page-image-aware entity recognition model in a [SageMaker Training Job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-training.html).

In this process:

- SageMaker will run the job on a dedicated, managed instance of type we choose (we'll use `ml.p*` or `ml.g*` GPU-accelerated types), allowing us to keep this notebook's resources modest and only pay for the seconds of GPU time the training job needs.
- The data as specified in the manifest files will be downloaded from Amazon S3.
- The bundle of scripts we provide (in `src/`) will be transparently uploaded to S3 and then run inside the specified SageMaker-provided [framework container](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html). There's no need for us to build our own container image or implement a serving stack for inference (although fully-custom containers are [also supported](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers.html)).
- Job hyperparameters will be passed through to our `src/` scripts as CLI arguments.
- SageMaker will analyze the logs from the job (i.e. `print()` or `logger` calls from our script) with the regular expressions specified in `metric_definitions`, to scrape structured timeseries metrics like loss and accuracy.
- When the job finishes, the contents of the `model` folder in the container will be automatically tarballed and uploaded to a `model.tar.gz` in Amazon S3.

You can also refer to [Hugging Face's own docs for training on SageMaker](https://huggingface.co/transformers/sagemaker.html) for more information and examples.

In [ ]:
print (sagemaker.get_execution_role() ) 

In [ ]:
from sagemaker.huggingface import HuggingFace as HuggingFaceEstimator

hyperparameters = {
    # 2023-05-24
    "model_name_or_path": "microsoft/layoutxlm-base",
    #"model_name_or_path": "microsoft/layoutlm-base-uncased",  
    #"model_name_or_path": "microsoft/layoutlmv2-base-uncased"

    # (See src/code/config.py for more info on script parameters)
    "annotation_attr": standard_label_field,
    "images_prefix": imgs_s3uri[len("s3://"):].partition("/")[2],
    "textract_prefix": textract_s3uri[len("s3://"):].partition("/")[2],
    "num_labels": len(fields) + 1,  # +1 for "other"

    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 4,

    "num_train_epochs": 20,
    "early_stopping_patience": 15,
    "metric_for_best_model": "eval_focus_else_acc_minus_one",
    "greater_is_better": "true",

    # Early stopping implies checkpointing every evaluation (epoch), so limit the total checkpoints
    # kept to avoid filling up disk:
    "save_total_limit": 10,
}

metric_definitions = [
    {"Name": "epoch", "Regex": util.training.get_hf_metric_regex("epoch")},
    {"Name": "learning_rate", "Regex": util.training.get_hf_metric_regex("learning_rate")},
    {"Name": "train:loss", "Regex": util.training.get_hf_metric_regex("loss")},
    {
        "Name": "validation:n_examples",
        "Regex": util.training.get_hf_metric_regex("eval_n_examples"),
    },
    {"Name": "validation:loss_avg", "Regex": util.training.get_hf_metric_regex("eval_loss")},
    {"Name": "validation:acc", "Regex": util.training.get_hf_metric_regex("eval_acc")},
    {
        "Name": "validation:n_focus_examples",
        "Regex": util.training.get_hf_metric_regex("eval_n_focus_examples"),
    },
    {
        "Name": "validation:focus_acc",
        "Regex": util.training.get_hf_metric_regex("eval_focus_acc"),
    },
    {
        "Name": "validation:target",
        "Regex": util.training.get_hf_metric_regex("eval_focus_else_acc_minus_one"),
    },
]

estimator = HuggingFaceEstimator(
    role=sagemaker.get_execution_role(),
    entry_point="train.py",
    source_dir="src",
    py_version=py_version,
    pytorch_version=pt_version,
    transformers_version=hf_version,
    image_uri=train_image_uri,  # Use customized training container image

    base_job_name="ws-xlm-cfpb-hf",
    output_path=f"s3://{bucket_name}/{bucket_prefix}trainjobs",

    instance_type="ml.p3.2xlarge",  # Could also consider ml.g4dn.xlarge
    instance_count=1,
    volume_size=80,

    debugger_hook_config=False,

    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    environment={
        # Required for our custom dataset loading code (which depends on tokenizer):
        "TOKENIZERS_PARALLELISM": "false",
    },
)

In [ ]:
print (thumbs_s3uri)
print ()
print (train_manifest_s3uri)
print ()
print (textract_s3uri)
print ()
print (test_manifest_s3uri)


Finally, the below cell will actually kick off the training job and stream logs from the running container.

> ℹ️ You'll also be able to check the status of the job in the [Training jobs page of the SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/jobs).

In [ ]:
inputs = {
    "images": thumbs_s3uri,  #2023-05-24 not needed for LayoutLM 
    "train": train_manifest_s3uri,
    "textract": textract_s3uri + "/",
    "validation": test_manifest_s3uri,
}
estimator.fit(inputs)

### One-click model deployment

Once the training job is complete, the model can be deployed to an endpoint via `estimator.deploy()` - specifying any extra parameters needed such as environment variables and, in this case, configurations for [Asynchronous Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html). Async inference endpoints in SageMaker can accept larger payloads and auto-scale down to 0 instances when not in use (if configured) - making them a useful option for many document processing use cases.

In [ ]:
training_job_name = estimator.latest_training_job.describe()["TrainingJobName"]
# Or:
# training_job_name = tuner.best_training_job()
# ws-xlm-cfpb-hf-2023-05-26-01-32-32-238
# ws-xlm-cfpb-hf-2023-05-26-01-32-32-238
# ws-xlm-cfpb-hf-2023-05-26-11-15-49-226
print (training_job_name)


predictor = estimator.deploy(
    # Avoid us accidentally deploying the same model twice by setting name per training job:
    endpoint_name=training_job_name + "-11",
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",  # Or try ml.m5.2xlarge
    image_uri=inf_image_uri,

    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),

    env={
        "PYTHONUNBUFFERED": "1",  # TODO: Disable once debugging is done
        "MMS_MAX_REQUEST_SIZE": str(100*1024*1024),  # Accept large payloads (docs)
        "MMS_MAX_RESPONSE_SIZE": str(100*1024*1024),  # Allow large responses
    },

    # Deploy in Asynchronous mode, to support large req/res payloads:
    async_inference_config=sagemaker.async_inference.AsyncInferenceConfig(
        output_path=f"s3://{config.model_results_bucket}",
        max_concurrent_invocations_per_instance=2,
        notification_config={
            "SuccessTopic": config.model_callback_topic_arn,
            "ErrorTopic": config.model_callback_topic_arn,
        },
    ),
)

If needed (for example, if your kernel crashes or restarts), you can also attach to previously deployed endpoints. Just look up the endpoint name from the SageMaker Console:

In [ ]:
# endpoint_name="xlm-cfpb-hf-2022-05-23-14-10-19-602"
# predictor = sagemaker.predictor_async.AsyncPredictor(
#     sagemaker.Predictor(
#         endpoint_name,
#         serializer=sagemaker.serializers.JSONSerializer(),
#         deserializer=sagemaker.deserializers.JSONDeserializer(),
#     ),
#     name=endpoint_name,
# )

---
## Extract clean input images on-demand

Just as we generated page thumbnail images to originally train our model, online inference should be able to generate these input features on-demand. In this example, the same code we previously used in a batch processing job has already been automatically deployed to a SageMaker inference endpoint for you. We can look up the endpoint name from the deployed stack parameters:

In [ ]:
preproc_endpoint_name = ssm.get_parameter(
    Name=config.thumbnail_endpoint_name_param,
)["Parameter"]["Value"]
print(f"Pre-created thumbnailer endpoint name:\n  {preproc_endpoint_name}")

The online thumbnail-generation endpoint accepts raw input documents (i.e. PDFs, images), and returns compressed arrays of page image data. From the name of the endpoint, you can configure I/O formats and connect from the notebook as shown below:

In [ ]:
try:
    desc = smclient.describe_endpoint(EndpointName=preproc_endpoint_name)
except smclient.exceptions.ClientError as e:
    if e.response.get("Error", {}).get("Message", "").startswith("Could not find"):
        desc = None  # Endpoint does not exist
    else:
        raise e  # Some other unknown issue

if desc is None:
    raise ValueError(
        "The configured thumbnailing endpoint does not exist in SageMaker. See the 'Optional "
        "Extras.ipynb' notebook for instructions to manually deploy the thumbnailer before "
        "continuing. Missing endpoint: %s" % preproc_endpoint_name
    )

preproc_predictor = sagemaker.predictor_async.AsyncPredictor(
    sagemaker.Predictor(
        preproc_endpoint_name,
        serializer=util.deployment.FileSerializer.from_filename("any.pdf"),
        deserializer=util.deployment.CompressedNumpyDeserializer(),
    ),
    name=preproc_endpoint_name,
)

So how would it look to test the endpoint from Python? Let's see an example:

In [ ]:
%%time

# Choose an input (document or image):
input_file = "data/raw/121 Financial Credit Union/Visa Credit Card Agreement.pdf"
#input_file = "data/imgs-clean/121 Financial Credit Union/Visa Credit Card Agreement-0001-1.png"

# Ensure de/serializers are correctly set up (since depends on input file type):
preproc_predictor.serializer = util.deployment.FileSerializer.from_filename(input_file)
preproc_predictor.deserializer = util.deployment.CompressedNumpyDeserializer()
# Duplication because of https://github.com/aws/sagemaker-python-sdk/issues/3100
preproc_predictor.predictor.serializer = preproc_predictor.serializer
preproc_predictor.predictor.deserializer = preproc_predictor.deserializer

# Run prediction:
print("Calling endpoint...")
resp = preproc_predictor.predict(input_file)
print(f"Got response of type {type(resp)}")

# Render result:
util.viz.draw_thumbnails_response(resp)

---
## Using the entity recognition model

Once the deployment is complete and a page thumbnail generator is ready, we're ready to test out inference on some documents!

### Making requests and rendering results

At a high level, the layout+language model accepts Textract-like JSON (e.g. as returned by [AnalyzeDocument](https://docs.aws.amazon.com/textract/latest/dg/API_AnalyzeDocument.html#API_AnalyzeDocument_ResponseSyntax) or [DetectDocumentText](https://docs.aws.amazon.com/textract/latest/dg/API_DetectDocumentText.html#API_DetectDocumentText_ResponseSyntax) APIs) and classifies each `WORD` [block](https://docs.aws.amazon.com/textract/latest/dg/API_Block.html) according to the entity classes we defined earlier: Returning the same JSON with additional fields added to indicate the predictions.

In addition (per the logic in [src/code/inference.py](src/code/inference.py)):

- To incorporate image features (for models that support them), requests can also include an `S3Thumbnails: { Bucket, Key }` object pointing to a thumbnailer endpoint response on S3.
- Instead of passing the (typically large and already-S3-resident) Amazon Textract JSON inline, an `S3Input: { Bucket, Key }` reference can be passed instead (and this is actually how the standard pipeline integration works).
- Output could also be redirected by passing an `S3Output: { Bucket, Key }` field in the request, but this is ignored and not needed on async endpoint deployments.
- `TargetPageNum` and `TargetPageOnly` fields can be specified to limit processing to a single page of the input document.

We can use utility functions to render these predictions as we did the manual annotations previously:

> ⏰ **Inference may take time in some cases:**
>
> - Although enabling thumbnails can increase demo inference time below by several seconds, the end-to-end pipeline generates these images in parallel with running Amazon Textract - so there's usually no significant impact in practice.
> - If you enabled **auto-scale-to-zero** on your your thumbnailer and/or model endpoint, you may see a cold-start of several minutes.

> ⚠️ **Check:** Because of the way the SageMaker Python SDK's [AsyncPredictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictor_async.html) emulates a synchronous `predict()` interface for async endpoints, you may find the notebook waits indefinitely instead of raising an error when something goes wrong. If an inference takes more than ~30s to complete, check the endpoint logs from your [SageMaker Console Endpoints page](https://console.aws.amazon.com/sagemaker/home?#/endpoints) to see if your request resulted in an error.

In [ ]:
import ipywidgets as widgets
import trp

# Enabling thumbnails can significantly increase inference time here, but can improve results for
# models that consume image features (like LayoutLMv2, XLM):
include_thumbnails = False

def predict_from_manifest_item(
    item,
    predictor,
    imgs_s3key_prefix=imgs_s3uri[len("s3://"):].partition("/")[2],
    raw_s3uri_prefix=raw_s3uri,
    textract_s3key_prefix=textract_s3uri[len("s3://"):].partition("/")[2],
    imgs_local_prefix="data/imgs-clean",
    textract_local_prefix="data/textracted",
    draw=True,
):
    paths = util.viz.local_paths_from_manifest_item(
        item,
        imgs_s3key_prefix,
        textract_s3key_prefix=textract_s3key_prefix,
        imgs_local_prefix=imgs_local_prefix,
        textract_local_prefix=textract_local_prefix,
    )

    if include_thumbnails:
        doc_textract_s3key = item["textract-ref"][len("s3://"):].partition("/")[2]
        doc_raw_s3uri = raw_s3uri_prefix + doc_textract_s3key[len(textract_s3key_prefix):].rpartition("/")[0]
        print(f"Fetching thumbnails for {doc_raw_s3uri}")
        thumbs_async = preproc_predictor.predict_async(input_path=doc_raw_s3uri)
        thumbs_bucket, _, thumbs_key = thumbs_async.output_path[len("s3://"):].partition("/")
        # Wait for the request to complete:
        thumbs_async.get_result(sagemaker.async_inference.WaiterConfig())
        req_extras = {"S3Thumbnails": {"Bucket": thumbs_bucket, "Key": thumbs_key}}
        print("Got thumbnails result")
    else:
        req_extras = {}

    result_json = predictor.predict({
        "S3Input": {"S3Uri": item["textract-ref"]},
        "TargetPageNum": item["page-num"],
        "TargetPageOnly": True,
        **req_extras,
    })

    if "Warnings" in result_json:
        for warning in result_json["Warnings"]:
            logger.warning(warning)
    result_trp = trp.Document(result_json)

    if draw:
        util.viz.draw_smgt_annotated_page(
            paths["image"],
            entity_classes,
            annotations=[],
            textract_result=result_trp,
            # Note that page_num should be item["page-num"] if we requested the full set of pages
            # from the model above:
            page_num=1,
        )
    return result_trp


widgets.interact(
    lambda ix: predict_from_manifest_item(test_examples[ix], predictor),
    ix=widgets.IntSlider(
        min=0,
        max=len(test_examples) - 1,
        step=1,
        value=0,
        description="Example:",
    )
)

### From token classification to entity detection

You may have noticed a slight mismatch: We're talking about extracting 'fields' or 'entities' from the document, but our model just classifies individual words. Going from words to entities assumes we're able to understand which words go "together" and what order they should be read in.

Fortunately, Amazon Textract helps us out with this too as the word blocks are already collected into `LINE`s.

For many straightforward applications, we can simply loop through the lines on a page and define an "entity detection" as a contiguous group of the same class - as below:

In [ ]:
print (test_examples[5])

In [ ]:
res = predict_from_manifest_item(
    test_examples[5],
    predictor,
    draw=False,
)

In [ ]:
other_cls = len(entity_classes)
prev_cls = other_cls
current_entity = ""

for page in res.pages:
    for line in page.lines:
        for word in line.words:
            pred_cls = word._block["PredictedClass"]
            if pred_cls != prev_cls:
                if prev_cls != other_cls:
                    print(f"----------\n{entity_classes[prev_cls]}:\n{current_entity}")
                prev_cls = pred_cls
                if pred_cls != other_cls:
                    current_entity = word.text
                else:
                    current_entity = ""
                continue
            current_entity = " ".join((current_entity, word.text))

Of course there may be some instances where this heuristic breaks down, but we still have access to all the position (and text) information from each `LINE` and `WORD` to write additional rules for reading order and separation if wanted.

---
## Setting up the end-to-end pipeline

### Integrating the entity detection model

So far we've demonstrated running entity detection requests from here in the notebook, but how can this model be integrated into the end-to-end document processing pipeline stack?

First, you'll identify the **endpoint name** of your deployed model and the **AWS Systems Manager Parameter** that configures the SageMaker endpoint parameter for the pipeline stack:

In [ ]:
print(f"Endpoint name:\n  {predictor.endpoint_name}")
print(f"\nEndpoint SSM param:\n  {config.sagemaker_endpoint_name_param}")

Finally, we'll update this SSM parameter to point to the deployed SageMaker endpoint.

The below code should do this for you automatically:

> ⚠️ **Note:** The [Lambda function](../pipeline/enrichment/fn-call-sagemaker/main.py) that calls your model from the OCR pipeline caches the endpoint name for a few minutes (`CACHE_TTL_SECONDS`) to reduce unnecessary ssm:GetParameter calls - so it may take a little time for an update here to take effect if you already processed a document recently.

In [ ]:
pipeline_endpoint_name = predictor.endpoint_name

print(f"Configuring pipeline with model: {pipeline_endpoint_name}")

ssm.put_parameter(
    Name=config.sagemaker_endpoint_name_param,
    Overwrite=True,
    Value=pipeline_endpoint_name,
)

Alternatively, you could open the [AWS Systems Manager Parameter Store console](https://console.aws.amazon.com/systems-manager/parameters/?tab=Table) and click on the *name* of the parameter to open its detail page, then the **Edit** button in the top right corner as shown below:

![](img/ssm-param-detail-screenshot.png "Screenshot of SSM parameter detail page showing Edit button")

From this screen you can manually set the **Value** of the parameter and save the changes.

Whether you updated the SSM parameters via code or the console, your the pre-processing and enrichment stages of your stack should now be configured to use your endpoints!

### Updating the pipeline entity definitions

As well as configuring the *enrichment* stage of the pipeline to reference the deployed version of the model, we need to configure the *post-processing* stage to match the model's **definition of entity/field types**.

The entity configuration is as we saved in the previous notebook, but the `annotation_guidance` attributes are not needed:

> ℹ️ **Note:** As well as the mapping from ID numbers (returned by the model) to human-readable class names, this configuration controls how the pipeline consolidates entity matches into "fields" of the document: E.g. choosing the "most likely" or "first" value between multiple detections, or setting up a multi-value field.

In [ ]:
pipeline_entity_config = json.dumps([f.to_dict(omit=["annotation_guidance"]) for f in fields], indent=2)
print(pipeline_entity_config)

As above, you *could* set this value manually in the SSM console for the parameter named as `EntityConfig`.

...But we can make the same update via code through the APIs:

In [ ]:
print(f"Setting pipeline entity configuration")
ssm.put_parameter(
    Name=config.entity_config_param,
    Overwrite=True,
    Value=pipeline_entity_config,
)

### Set up online review with Amazon Augmented AI (A2I)

Whereas our original batch annotation used the [built-in](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-task-types.html) image bounding box / object detection task UI, a custom task template is provided for online review.

Since the template is built using a web framework (VueJS), we'll need to install some extra dependencies to enable building it:

In [ ]:
!cd review && npm install

Then, build the UI HTML template from source:

In [ ]:
!cd review && npm run build
ui_template_file = "review/dist/index.html"

Next, upload the built file as an A2I human review task UI:

In [ ]:
with open(ui_template_file, "r") as f:
    create_template_resp = smclient.create_human_task_ui(
        HumanTaskUiName="fields-validation-1",  # (Can change this name as you like)
        UiTemplate={"Content": f.read()},
    )

task_template_arn = create_template_resp["HumanTaskUiArn"]
print(f"Created A2I task template:\n{task_template_arn}")

We already defined a "team" for tasks to be routed to above, for SageMaker Ground Truth, and can re-use that team for the online review flow.

To finish setting up the workflow itself, we need 2 more pieces of information:

- The **location in S3** where review outputs should be stored
- An appropriate **execution role** which will give the A2I workflow to read input documents and write review results.

These are determined by the **OCR pipeline solution stack**, because the reviews bucket is created by the pipeline with event triggers to resume the next stage when reviews are uploaded.

The code below should be able to look up these parameters for you automatically:

In [ ]:
reviews_bucket_name = config.pipeline_reviews_bucket_name
print(reviews_bucket_name)
reviews_role_arn = config.a2i_execution_role_arn
print(reviews_role_arn)

Alternatively, you may **find** your pipeline solution stack from the [AWS CloudFormation Console](https://console.aws.amazon.com/cloudformation/home?#/stacks) and click through to the stack detail page. From the **Outputs** tab, you should see the `A2IHumanReviewBucketName` and `A2IHumanReviewExecutionRoleArn` values as shown below.

(You may also note the `A2IHumanReviewFlowParamName`, which we'll use in the next section)

![](img/cfn-stack-outputs-a2i.png "CloudFormation stack outputs for OCR pipeline")

Once these values are populated, you're ready to create your review workflow by running the code below.

Note that you can also manage flows via the [A2I Human Review Workflows Console](https://console.aws.amazon.com/a2i/home?#/human-review-workflows/).

In [ ]:
create_flow_resp = smclient.create_flow_definition(
    FlowDefinitionName="ocr-fields-validation-1",  # (Can change this name as you like)
    HumanLoopConfig={
        "WorkteamArn": workteam_arn,
        "HumanTaskUiArn": task_template_arn,
        "TaskTitle": "Review OCR Field Extractions",
        "TaskDescription": "Review and correct credit card agreement field extractions",
        "TaskCount": 1,  # One reviewer per item
        "TaskAvailabilityLifetimeInSeconds": 60 * 60,  # Availability timeout
        "TaskTimeLimitInSeconds": 60 * 60,  # Working timeout
    },
    OutputConfig={
        "S3OutputPath": f"s3://{reviews_bucket_name}/reviews",
    },
    RoleArn=reviews_role_arn,
)

print(f"Created review workflow:\n{create_flow_resp['FlowDefinitionArn']}")

Finally, when the human review flow is created and registered, we can configure the document pipeline to use it - similarly to our SageMaker endpoint and entity configuration:

In [ ]:
print(f"Configuring pipeline with review workflow: {create_flow_resp['FlowDefinitionArn']}")

ssm = boto3.client("ssm")
ssm.put_parameter(
    Name=config.a2i_review_flow_arn_param,
    Overwrite=True,
    Value=create_flow_resp["FlowDefinitionArn"],
)

Alternatively through the console, you would follow these steps:

▶️ **Check** the `A2IHumanReviewFlowParamName` output of your OCR pipeline stack in [CloudFormation](https://console.aws.amazon.com/cloudformation/home?#/stacks) (as we did above)

▶️ **Open** the [AWS Systems Manager Parameter Store console](https://console.aws.amazon.com/systems-manager/parameters/?tab=Table) and **find the review flow parameter in the list**.

▶️ **Click** on the name of the parameter to open its detail page, and then on the **Edit** button in the top right corner. Set the value to the **workflow ARN** (see previous code cell in this notebook) and save the changes.

![](img/ssm-a2i-param-detail.png "Screenshot of SSM parameter detail page for human workflow")

---
## Final testing

Your OCR pipeline should now be fully functional! Let's try it out:

▶️ **Log in** to the labelling portal (URL available from the [SageMaker Ground Truth Workforces Console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-workforces) for your correct AWS Region)

![](img/smgt-find-workforce-url.png "Screenshot of SMGT console with workforce login URL")

▶️ **Upload** one of the sample documents to your pipeline's input bucket in Amazon S3, either using the code snippets below or drag and drop in the [Amazon S3 Console](https://console.aws.amazon.com/s3/)

In [ ]:
pdfpaths = []
for currpath, dirs, files in os.walk("data/raw"):
    if "/." in currpath or "__" in currpath:
        continue
    pdfpaths += [
        os.path.join(currpath, f) for f in files
        if f.lower().endswith(".pdf")
    ]
pdfpaths = sorted(pdfpaths)

In [ ]:
test_filepath = pdfpaths[14]
test_s3uri = f"s3://{config.pipeline_input_bucket_name}/{test_filepath}"

!aws s3 cp '{test_filepath}' '{test_s3uri}'

▶️ **Open up** your "Processing Pipeline" state machine in the [AWS Step Functions Console](https://console.aws.amazon.com/states/home?#/statemachines)

After a few seconds you should find that a Step Function execution is automatically triggered and (since we enabled so many fields that at least one is always missing) the example is eventually forwarded for human review in A2I.

As you'll see from the `ModelResult` field in your final *Step Output*, this pipeline produces a rich but usefully-structured output - with good opportunities for onward integration into further Step Functions steps or external systems. You can find more information and sample solutions for integrating AWS Step Functions in the [Step Functions Developer Guide](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html).

![](img/sfn-statemachine-success.png "Screenshot of successful Step Function execution with output JSON")

## Conclusion

In this worked example we showed how advanced, open-source language processing models specifically tailored for document understanding can be integrated with [Amazon Textract](https://aws.amazon.com/textract/): providing a trainable, ML-driven framework for tackling more niche or complex requirements where Textract's [built-in structure extraction tools](https://aws.amazon.com/textract/features/) may not fully solve the challenges out-of-the-box.

The underlying principle of the model - augmenting multi-task neural text processing architectures with positional data - is highly extensible, with potential to tackle a wide range of use cases where joint understanding of the content and presentation of text can deliver better results than considering text alone.

We demonstrated how an end-to-end process automation pipeline applying this technology might look: Developing and deploying the model with [Amazon SageMaker](https://aws.amazon.com/sagemaker/), building a serverless workflow with [AWS Step Functions](https://aws.amazon.com/step-functions/) and [AWS Lambda](https://aws.amazon.com/lambda/), and driving quality with human review of low-confidence documents through [Amazon Augmented AI](https://aws.amazon.com/augmented-ai/).

Thanks for following along, and for more information, don't forget to check out:

- The other published [Amazon Textract Examples](https://docs.aws.amazon.com/textract/latest/dg/other-examples.html) listed in the [Textract Developer Guide](https://docs.aws.amazon.com/textract/latest/dg/what-is.html)
- The extensive repository of [Amazon SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples) and usage documentation in the [SageMaker Python SDK User Guide](https://sagemaker.readthedocs.io/en/stable/) - as well as the [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/index.html)
- The wide range of other open algorithms and models published by [HuggingFace Transformers](https://huggingface.co/transformers/), and their specific documentation on [using the library with SageMaker](https://huggingface.co/transformers/sagemaker.html)
- The conversational AI and NLP area (and others) of Amazon's own [Amazon.Science](https://www.amazon.science/conversational-ai-natural-language-processing) blog

Happy building!

In [ ]:
# shut down end point after use so we don't incur needless charges. 
predictor.delete_endpoint(delete_endpoint_config=True)